# ICS project

## 0) Context

Sophie wants to study ChIP-Seq samples from 2 publications (organism = *Mus musculus*, mm9). <br>
Each publication as an associated GEO dataset where BIGWIGs are already available. <br>

Goal for each sample : <br>
*Nb* : Work is done for 4 regions of interest : smu, sgamma1, sgamma3, 3'RR. Sophie expects 3'RR & sgamma1 to respond to activation (but maybe not for the same marks). Sgamma3 should be ~ negative control <br>
> - display density tracks. <br>
> - report underlying values in spreadsheets. <br>
> - add annotations (hs & ls regions, CSReport breakpoints counts). <br>
> - highlight up/down marks or differentially expressed marks. <br>

## 1) Get inputs

Discussion by mail with Seolkyoung Jung, author in the publication

Send me resting and stimulated input files via Globus

## 2) Get fastq

### 2.1) Install aspera

https://www.biostars.org/p/325010/

In [ ]:
wget https://download.asperasoft.com/download/sw/connect/3.8.0/ibm-aspera-connect-3.8.0.158555-linux-g2.12-64.tar.gz 
## Decompress:
tar zxvf ibm-aspera-connect-3.8.0.158555-linux-g2.12-64.tar.gz 
## Make it executable:
chmod +x ibm-aspera-connect-3.8.0.158555-linux-g2.12-64.sh
## and run the installer:
./ibm-aspera-connect-3.8.0.158555-linux-g2.12-64.sh

### 2.2) Download fastq

In [ ]:
#Only select Study accession, experiment title and fastq ftp
#Remove manually line I don't want

awk 'FS="\t", OFS="\t" { gsub("ftp.sra.ebi.ac.uk", "era-fasp@fasp.sra.ebi.ac.uk:"); print }' PRJNA324130.txt | cut -f3 | awk -F ";" 'OFS="\n" {print $1, $2}' | awk NF | awk 'NR > 1, OFS="\n" {print "ascp -QT -l 300m -P33001 -i /home/bastien/.aspera/connect/etc/asperaweb_id_dsa.openssh" " " $1 " ."}' > download_PRJNA324130.txt

awk -F'[;/:]' '{print $10 "\t" $3}' PRJNA324130.txt > metadata_PRJNA324130.txt

chmod 755 /home/bastien/.aspera/connect/etc/asperaweb_id_dsa.openssh

#Add to bashrc
export PATH=$PATH:/home/bastien/.aspera/connect/bin

cd /media/sf_raid/Data/ICS/fastq/Data

while read LIST; do $LIST; done < ../download_PRJNA324130.txt

#Create a metadata marks you want, then rename the files accordingly
awk '{system("mv " $1 ".fastq.gz " $2 ".fastq.gz")}' < ../metadata_PRJNA324130_marksOnly.txt

#Remove by hands the files unrenamed

## 3) FastQC

In [ ]:
cd /media/sf_raid/Data/ICS/fastq/Data
for file in *; do fastqc $file; done

cd /media/sf_raid/Data/ICS/fastq/Input
for file in *; do fastqc $file; done

## 4) MultiQC

In [ ]:
cd /media/sf_raid/Data/ICS/fastq/Data
multiqc -o ../FastQC *zip

## 5) Download reference

In [ ]:
cd /media/sf_raid/Data/ICS/fastq/index
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M19/GRCm38.primary_assembly.genome.fa.gz
#wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M19/gencode.vM19.primary_assembly.annotation.gtf.gz
#Upload on genotoul

## 6) BWA Index

In [ ]:
$bwa/bwa index -a bwtsw $ics/Index/GRCm38.primary_assembly.genome.fa

## 7) BWA Alignment

In [ ]:
for id in 'aB_wt_H3K9me2' 'rB_wt_ChIP_input' 'rB_wt_H3K36me3'; do
$bwa/bwa mem -t $nt -M $ics/Index/GRCm38.primary_assembly.genome.fa $ics/Data/$id.fastq.gz > $ics/Data/Aligned/$id.sam
$samtools/samtools view -Sb $ics/Data/Aligned/$id.sam $ics/Data/Aligned/$id.bam
$samtools/samtools sort $ics/Data/Aligned/$id.sam > $ics/Data/Aligned/$id.bam
$samtools/samtools index $ics/Data/Aligned/$id.bam
rm $ics/Data/Aligned/$id.sam
done;

## 8) Install Anaconda

In [ ]:
#Install Anaconda
#https://www.anaconda.com/download/
bash Miniconda3-latest-Linux-x86_64.sh

#Conda in bashrc
export PATH="/home/bastien/anaconda3/bin/:$PATH"

#Install misopy
conda install -c bioconda macs2

#Set python 2.7 env
conda create -n python2 python=2.7 anaconda
source activate python2
#source desactivate

## 9) Coverage normalized

In [ ]:
# OTHER TESTS

cd /media/sf_raid/Data/ICS/
mkdir bedgraph_normalized
#RPGC = reads per genomic content (1x normalization); Mapped reads are considered after blacklist filtering
#RPGC (per bin) = number of reads per bin / scaling factor for 1x average coverage.
#This scaling factor, in turn, is determined from the sequencing depth:
#(total number of mapped reads * fragment length) / effective genome size
#The scaling factor used is the inverse of the sequencing depth computed for the sample to match the 1x coverage
# This option requires --effectiveGenomeSize.

for file in aligned/*.bam; do
bamCoverage --bam $file --binSize 100 --outFileName bedgraph_normalized/$(basename ${file%%.*}).normalized.bdg --outFileFormat bedgraph --region chr12 --normalizeUsing RPGC --effectiveGenomeSize 2652783500 --ignoreDuplicates
done;

## 10) Mean standard deviation plot

In [ ]:
cd /media/sf_raid/Data/ICS/

paste bedgraph_normalized/*.bdg | awk 'FS=OFS="\t"{printf $1"_"$2"_"$3; for (i=4; i <= NF; i+=4) printf("\t"$i); print FL }' > merged.normalized.bdg
mv merged.normalized.bdg bedgraph_normalized/
sed -i '1s/^/\taB24h_wt_ChIP_input\taB_wt_H2AZ\taB_wt_H3K27Ac\taB_wt_H3K36me1\taB_wt_H3K36me2\taB_wt_H3K36me3\taB_wt_H3K4me1\taB_wt_H3K4me3\taB_wt_H3K9me1\taB_wt_H3K9me2\taB_wt_H3K9me3\taB_wt_H4K16Ac\taB_wt_H4K20me1\taB_wt_H4K20me3\trB_wt_ChIP_input\trB_wt_H2AZ\trB_wt_H3K27Ac\trB_wt_H3K36me1\trB_wt_H3K36me2\trB_wt_H3K36me3\trB_wt_H3K4me1\trB_wt_H3K4me3\trB_wt_H3K9me1\trB_wt_H3K9me2\trB_wt_H3K9me3\trB_wt_H4K16Ac\trB_wt_H4K20me1\trB_wt_H4K20me3\n/' bedgraph_normalized/merged.normalized.bdg
echo -e '\tcondition\ttype\naB24h_wt_ChIP_input\ttreated\tsingle-read\naB_wt_H2AZ\ttreated\tsingle-read\naB_wt_H3K27Ac\ttreated\tsingle-read\naB_wt_H3K36me1\ttreated\tsingle-read\naB_wt_H3K36me2\ttreated\tsingle-read\naB_wt_H3K36me3\ttreated\tsingle-read\naB_wt_H3K4me1\ttreated\tsingle-read\naB_wt_H3K4me3\ttreated\tsingle-read\naB_wt_H3K9me1\ttreated\tsingle-read\naB_wt_H3K9me2\ttreated\tsingle-read\naB_wt_H3K9me3\ttreated\tsingle-read\naB_wt_H4K16Ac\ttreated\tsingle-read\naB_wt_H4K20me1\ttreated\tsingle-read\naB_wt_H4K20me3\ttreated\tsingle-read\nrB_wt_ChIP_input\tuntreated\tsingle-read\nrB_wt_H2AZ\tuntreated\tsingle-read\nrB_wt_H3K27Ac\tuntreated\tsingle-read\nrB_wt_H3K36me1\tuntreated\tsingle-read\nrB_wt_H3K36me2\tuntreated\tsingle-read\nrB_wt_H3K36me3\tuntreated\tsingle-read\nrB_wt_H3K4me1\tuntreated\tsingle-read\nrB_wt_H3K4me3\tuntreated\tsingle-read\nrB_wt_H3K9me1\tuntreated\tsingle-read\nrB_wt_H3K9me2\tuntreated\tsingle-read\nrB_wt_H3K9me3\tuntreated\tsingle-read\nrB_wt_H4K16Ac\tuntreated\tsingle-read\nrB_wt_H4K20me1\tuntreated\tsingle-read\nrB_wt_H4K20me3\tuntreated\tsingle-read' > bedgraph_normalized/design_MA_plot.txt

mkdir bed_high_coverage
mkdir meanSDplot
/media/sf_raid/Projects/ICS/meanSDplot.R

## 11) High counts bam files

In [ ]:
cd /media/sf_raid/Data/ICS/
sed -i 's/"//g;s/_/\t/g' bed_high_coverage/*.bed

mkdir aligned_best

for file in bed_high_coverage/*.bed; do
bedtools intersect -abam aligned/aB_wt_$(basename ${file%%.*}).bam -b $file > aligned_best/aB_wt_$(basename ${file%%.*}).bam;
bedtools intersect -abam aligned/rB_wt_$(basename ${file%%.*}).bam -b $file > aligned_best/rB_wt_$(basename ${file%%.*}).bam;
done;

## 12) Peak calling

In [ ]:
cd /media/sf_raid/Data/ICS/
mkdir peakcall

#Sharp peaks
for mark in 'H2AZ' 'H3K4me3' 'H3K27Ac' 'H3K36me2' 'H4K16Ac' 'H4K20me3'; do
macs2 callpeak -t aligned_best/aB_wt_$mark.bam -c aligned/aB24h_wt_ChIP_input.bam -g mm --outdir peakcall --nomodel --extsize 147 --mfold 5 50 -B -n aB_wt_$mark
macs2 callpeak -t aligned_best/rB_wt_$mark.bam -c aligned/rB_wt_ChIP_input.bam -g mm --outdir peakcall --nomodel --extsize 147 --mfold 5 50 -B -n rB_wt_$mark
done;

#Broad peaks
for mark in 'H3K4me1' 'H3K9me1' 'H3K9me2' 'H3K9me3' 'H3K36me1' 'H3K36me3' 'H4K20me1'; do
macs2 callpeak --broad -t aligned_best/aB_wt_$mark.bam -c aligned/aB24h_wt_ChIP_input.bam -g mm --outdir peakcall --nomodel --extsize 147 --mfold 5 50 -B -n aB_wt_$mark
macs2 callpeak --broad -t aligned_best/rB_wt_$mark.bam -c aligned/rB_wt_ChIP_input.bam -g mm --outdir peakcall --nomodel --extsize 147 --mfold 5 50 -B -n rB_wt_$mark
done;

## 13) Normalization & Differential

In [ ]:
cd /media/sf_raid/Data/ICS/
mkdir bdgdiff
cd aligned
# Get the number of mapped reads
for file in *.bam; do echo $file; samtools view -F 0x904 -c $file; done;
#aB24h_wt_ChIP_input.bam
#27087847
#aB_wt_H2AZ.bam
#42559264
#aB_wt_H3K27Ac.bam
#27207808
#aB_wt_H3K36me1.bam
#41781293
#aB_wt_H3K36me2.bam
#47105589
#aB_wt_H3K36me3.bam
#50091682
#aB_wt_H3K4me1.bam
#7569095
#aB_wt_H3K4me3.bam
#16968946
#aB_wt_H3K9me1.bam
#41801665
#aB_wt_H3K9me2.bam
#67600519
#aB_wt_H3K9me3.bam
#106257991
#aB_wt_H4K16Ac.bam
#51951324
#aB_wt_H4K20me1.bam
#21442330
#aB_wt_H4K20me3.bam
#33188484
#rB_wt_ChIP_input.bam
#24613767
#rB_wt_H2AZ.bam
#27689824
#rB_wt_H3K27Ac.bam
#27349088
#rB_wt_H3K36me1.bam
#38353446
#rB_wt_H3K36me2.bam
#40007633
#rB_wt_H3K36me3.bam
#66656791
#rB_wt_H3K4me1.bam
#6048520
#rB_wt_H3K4me3.bam
#17356485
#rB_wt_H3K9me1.bam
#58926859
#rB_wt_H3K9me2.bam
#49074294
#rB_wt_H3K9me3.bam
#53740072
#rB_wt_H4K16Ac.bam
#38216075
#rB_wt_H4K20me1.bam
#46179820
#rB_wt_H4K20me3.bam
#42349510

#Sharp peaks
#'H2AZ' 'H3K4me3' 'H3K27Ac' 'H3K36me2' 'H4K16Ac' 'H4K20me3'
macs2 bdgdiff --t1 peakcall/aB_wt_H2AZ_treat_pileup.bdg --t2 peakcall/rB_wt_H2AZ_treat_pileup.bdg --c1 peakcall/aB_wt_H2AZ_control_lambda.bdg --c2 peakcall/rB_wt_H2AZ_control_lambda.bdg --d1 42559264 --d2 27689824 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H2AZ.diff.bdg rB_wt_H2AZ.diff.bdg common_wt_H2AZ.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K4me3_treat_pileup.bdg --t2 peakcall/rB_wt_H3K4me3_treat_pileup.bdg --c1 peakcall/aB_wt_H3K4me3_control_lambda.bdg --c2 peakcall/rB_wt_H3K4me3_control_lambda.bdg --d1 16968946 --d2 17356485 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K4me3.diff.bdg rB_wt_H3K4me3.diff.bdg common_wt_H3K4me3.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K27Ac_treat_pileup.bdg --t2 peakcall/rB_wt_H3K27Ac_treat_pileup.bdg --c1 peakcall/aB_wt_H3K27Ac_control_lambda.bdg --c2 peakcall/rB_wt_H3K27Ac_control_lambda.bdg --d1 27207808 --d2 27349088 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K27Ac.diff.bdg rB_wt_H3K27Ac.diff.bdg common_wt_H3K27Ac.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K36me2_treat_pileup.bdg --t2 peakcall/rB_wt_H3K36me2_treat_pileup.bdg --c1 peakcall/aB_wt_H3K36me2_control_lambda.bdg --c2 peakcall/rB_wt_H3K36me2_control_lambda.bdg --d1 47105589 --d2 40007633 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K36me2.diff.bdg rB_wt_H3K36me2.diff.bdg common_wt_H3K36me2.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H4K16Ac_treat_pileup.bdg --t2 peakcall/rB_wt_H4K16Ac_treat_pileup.bdg --c1 peakcall/aB_wt_H4K16Ac_control_lambda.bdg --c2 peakcall/rB_wt_H4K16Ac_control_lambda.bdg --d1 51951324 --d2 38216075 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H4K16Ac.diff.bdg rB_wt_H4K16Ac.diff.bdg common_wt_H4K16Ac.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H4K20me3_treat_pileup.bdg --t2 peakcall/rB_wt_H4K20me3_treat_pileup.bdg --c1 peakcall/aB_wt_H4K20me3_control_lambda.bdg --c2 peakcall/rB_wt_H4K20me3_control_lambda.bdg --d1 33188484 --d2 42349510 --min-len 150 --max-gap 100 --outdir bdgdiff -o aB_wt_H4K20me3.diff.bdg rB_wt_H4K20me3.diff.bdg common_wt_H4K20me3.diff.bdg


#broad peaks
# 'H3K4me1' 'H3K9me1' 'H3K9me2' 'H3K9me3' 'H3K36me1' 'H3K36me3' 'H4K20me1'
macs2 bdgdiff --t1 peakcall/aB_wt_H3K4me1_treat_pileup.bdg --t2 peakcall/rB_wt_H3K4me1_treat_pileup.bdg --c1 peakcall/aB_wt_H3K4me1_control_lambda.bdg --c2 peakcall/rB_wt_H3K4me1_control_lambda.bdg --d1 7569095 --d2 6048520 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K4me1.diff.bdg rB_wt_H3K4me1.diff.bdg common_wt_H3K4me1.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K9me1_treat_pileup.bdg --t2 peakcall/rB_wt_H3K9me1_treat_pileup.bdg --c1 peakcall/aB_wt_H3K9me1_control_lambda.bdg --c2 peakcall/rB_wt_H3K9me1_control_lambda.bdg --d1 41801665 --d2 58926859 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K9me1.diff.bdg rB_wt_H3K9me1.diff.bdg common_wt_H3K9me1.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K9me2_treat_pileup.bdg --t2 peakcall/rB_wt_H3K9me2_treat_pileup.bdg --c1 peakcall/aB_wt_H3K9me2_control_lambda.bdg --c2 peakcall/rB_wt_H3K9me2_control_lambda.bdg --d1 67600519 --d2 49074294 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K9me2.diff.bdg rB_wt_H3K9me2.diff.bdg common_wt_H3K9me2.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K9me3_treat_pileup.bdg --t2 peakcall/rB_wt_H3K9me3_treat_pileup.bdg --c1 peakcall/aB_wt_H3K9me3_control_lambda.bdg --c2 peakcall/rB_wt_H3K9me3_control_lambda.bdg --d1 106257991 --d2 53740072 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K9me3.diff.bdg rB_wt_H3K9me3.diff.bdg common_wt_H3K9me3.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K36me1_treat_pileup.bdg --t2 peakcall/rB_wt_H3K36me1_treat_pileup.bdg --c1 peakcall/aB_wt_H3K36me1_control_lambda.bdg --c2 peakcall/rB_wt_H3K36me1_control_lambda.bdg --d1 41781293 --d2 38353446 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K36me1.diff.bdg rB_wt_H3K36me1.diff.bdg common_wt_H3K36me1.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H3K36me3_treat_pileup.bdg --t2 peakcall/rB_wt_H3K36me3_treat_pileup.bdg --c1 peakcall/aB_wt_H3K36me3_control_lambda.bdg --c2 peakcall/rB_wt_H3K36me3_control_lambda.bdg --d1 50091682 --d2 66656791 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H3K36me3.diff.bdg rB_wt_H3K36me3.diff.bdg common_wt_H3K36me3.diff.bdg
macs2 bdgdiff --t1 peakcall/aB_wt_H4K20me1_treat_pileup.bdg --t2 peakcall/rB_wt_H4K20me1_treat_pileup.bdg --c1 peakcall/aB_wt_H4K20me1_control_lambda.bdg --c2 peakcall/rB_wt_H4K20me1_control_lambda.bdg --d1 21442330 --d2 46179820 --min-len 400 --max-gap 100 --outdir bdgdiff -o aB_wt_H4K20me1.diff.bdg rB_wt_H4K20me1.diff.bdg common_wt_H4K20me1.diff.bdg

#Remove header and some column to be display on IGV
cd /media/sf_raid/Data/ICS
for file in bdgdiff/*; do
#Remove header (NH - No Header)
tail -n +2 $file > bdgdiff/NH_$(basename $file);
#Prepare to visualize
awk '{print $1"\t"$2"\t"$3"\t"$5}' bdgdiff/NH_$(basename $file) > bdgdiff/BED_NH_$(basename $file);
done;

## 14) Check locus

In [ ]:
#chr12	113363298	113365156	gamma3
#chr12	113330756	113338695	gamma1
#chr12	113423027	113426676	Smu
#chr12	113225832	113255223	3'RR

#Split peaks by locis
cd /media/sf_raid/Data/ICS/
mkdir peaks_locus
#Minus/Plus 1000 bases to catch peaks in promoters...
for file in bdgdiff/BED_NH_*; do
echo -e "chr12\t113362298\t113366156\tgamma3" | bedtools intersect -a $file -b - -nonamecheck > peaks_locus/$(basename ${file%%.*})_Sg3.locus.diff.bdg | cut -c8-;
echo -e "chr12\t113329756\t113339695\tgamma1" | bedtools intersect -a $file -b - -nonamecheck > peaks_locus/$(basename ${file%%.*})_Sg1.locus.diff.bdg | cut -c8-;
echo -e "chr12\t113422027\t113427676\tSmu" | bedtools intersect -a $file -b - -nonamecheck > peaks_locus/$(basename ${file%%.*})_Smu.locus.diff.bdg | cut -c8-;
echo -e "chr12\t113224832\t113256223\t3RR" | bedtools intersect -a $file -b - -nonamecheck > peaks_locus/$(basename ${file%%.*})_3RR.locus.diff.bdg | cut -c8-;
done;

python /media/sf_raid/Projects/ICS/final_table.py -r peaks_locus -o final_table.csv

## 15) Fold change per region

In [ ]:
cd /media/sf_raid/Data/ICS/
for file in aligned/*.bam; do
TmpScale=$(bc <<< "scale=6;10000000/$(samtools view -F 0x904 -c $file)");
nbReadsMapped=$(samtools view $file chr12:113362298-113366156 | wc -l);
echo -e "$(basename ${file%.*})\tSgamma3\t$(bc <<<$nbReadsMapped*$TmpScale)" >> reads_mapped_regions_normalized.txt;
nbReadsMapped=$(samtools view $file chr12:113329756-113339695 | wc -l);
echo -e "$(basename ${file%.*})\tSgamma1\t$(bc <<<$nbReadsMapped*$TmpScale)" >> reads_mapped_regions_normalized.txt;
nbReadsMapped=$(samtools view $file chr12:113422027-113427676 | wc -l);
echo -e "$(basename ${file%.*})\tSmu\t$(bc <<<$nbReadsMapped*$TmpScale)" >> reads_mapped_regions_normalized.txt;
nbReadsMapped=$(samtools view $file chr12:113224832-113256223 | wc -l);
echo -e "$(basename ${file%.*})\t3RR\t$(bc <<<$nbReadsMapped*$TmpScale)" >> reads_mapped_regions_normalized.txt;
done;

mkdir FC_plot
./media/sf_raid/Projects/ICS/FCplot.R